# Vector Search Basics

# Create the Vector Search Index
Create a schema and load the table with data, including embeddings we generate through the OpenAI Embedding API.

## Imports

In [12]:
from cassandra.cluster import Cluster
from cassandra.cluster import EXEC_PROFILE_DEFAULT
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
import pandas as pd

## Keys & Environment Variables

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

# Astra DB
ASTRA_DB_KEYSPACE = os.environ['ASTRA_DB_KEYSPACE']
ASTRA_DB_SECURE_BUNDLE_PATH = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']

# OpenAI Token
openai_api_key = os.environ['OPENAI_API_KEY']
openai.api_key = openai_api_key

## Select a model to compute embeddings

Embeddings are numerical representations of concepts converted to number sequences, which make it easy for computers to understand the relationships between those concepts.

This new embedding model from openAI - `text-embedding-ada-002` - replaces five separate models for text search, text similarity, and code search, and outperforms our previous most capable model, Davinci, at most tasks, while being priced 99.8% lower.

In [ ]:
model_id = "text-embedding-ada-002"

## Connect to Astra DB

In [14]:
cloud_config= {
  'secure_connect_bundle': ASTRA_DB_SECURE_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider('token', ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
#session.set_keyspace('vector_search_basics')
session.set_keyspace('lawchat')
session

## Database Schema

> **Note:** The following blocks only need be run when you create the schema. Otherwise use them at your dicretion.

Note the data type `vector` in the schema below.

### Drop Schema

> **Note:** Only run this block when you want to DROP the schema.

In [ ]:
# only use this to DROP the schema
session.execute(f"""DROP INDEX IF EXISTS openai_desc""")
session.execute(f"""DROP INDEX IF EXISTS minilm_desc""")
session.execute(f"""DROP TABLE IF EXISTS products_table""")

### Create Schema

> **Note:** Only run this block when you want to CREATE the schema.

In [ ]:
# CREATE the schema

session.execute(f"""CREATE TABLE IF NOT EXISTS products_table
(product_id int,
 chunk_id int,

 product_name text,
 description text,
 price text,

 openai_description_embedding vector<float, 1536>,
 minilm_description_embedding vector<float, 384>,

 PRIMARY KEY (product_id, chunk_id))""")

# # Create Index
session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS openai_desc ON products_table (openai_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")
session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS minilm_desc ON products_table (minilm_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")


## Create embeddings and Store in DB 

### Read CSV file

In [ ]:
products_list = pd.read_csv('ProductDataset.csv')
products_list

### Create embeddings and insert into database

> **Note:** You only need to run this block once after creating the database schema.

In [ ]:
# Create Embedding for each conversation row, save them to the database
for id, row in products_list.iterrows():

  # break Description data into chunks of 2500 characters
  text_chunk_length = 2500
  text_chunks = [row.description[i:i + text_chunk_length] for i in range(0, len(row.description), text_chunk_length)]
  
  for chunk_id, chunk in enumerate(text_chunks):
    
    # Append Price to Description Data 
    pricevalue = row.price if isinstance(row.price, str) else ""
    full_chunk = f"{chunk} price: {pricevalue}"

    # Create an embedding using OpenAI API
    embedding = openai.Embedding.create(input=full_chunk, model=model_id)['data'][0]['embedding']

    # Insert Data and Embedding into database
    query = SimpleStatement(
                f"""
                INSERT INTO products_table
                (product_id, chunk_id, product_name, description, price, openai_description_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )
    session.execute(query, (row.product_id, chunk_id, row.product_name, row.description, pricevalue, embedding))



---


# Use the index

In the steps up to this point, we have been creating a schema and loading the table with data, including embeddings we generated through the OpenAI Embedding API.
Now we are going to query that table and use the results to give ChatGPT some context to support it's response.

## Convert a query string into a text embedding to use as part of the query

This is where the real fun starts.  Provide a question or request to be used as the query.  The source sample database is mostly consumer electronics and appliances, so imagine you're talking to a customer service rep at Best Buy or another electronics store.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [20]:
customer_input = "who is the chemist"
embedding = openai.Embedding.create(input=customer_input, model=model_id)['data'][0]['embedding']
display(embedding)

[0.01919185183942318,
 0.030864622443914413,
 -0.007852590642869473,
 -0.01625092141330242,
 0.00016770132060628384,
 0.030318882316350937,
 0.005904602352529764,
 -0.029257722198963165,
 -0.008072402328252792,
 -0.015871934592723846,
 0.01738787814974785,
 -0.011854683049023151,
 -0.01993466541171074,
 0.016159964725375175,
 -0.0022777060512453318,
 -0.012999220751225948,
 0.03450288996100426,
 0.009429172612726688,
 -0.014136179350316525,
 -0.010384216904640198,
 -0.014628861099481583,
 0.0003174007870256901,
 0.008095141500234604,
 -0.0018674536840990186,
 0.008959230035543442,
 0.004877550061792135,
 0.03153163939714432,
 -0.03941454738378525,
 -0.01858547516167164,
 -0.014045222662389278,
 -0.011089131236076355,
 -0.004165056627243757,
 -0.03992996737360954,
 -0.044871944934129715,
 -0.01769106835126877,
 -0.008413489907979965,
 0.005218637641519308,
 0.0006475923582911491,
 0.019146373495459557,
 -0.00018783495761454105,
 0.009398853406310081,
 0.010816261172294617,
 -0.002332658

## Find the top 5 results using ANN Similarity

Let's take a look at what a query against a vector index could look like.  The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

In [21]:
query = SimpleStatement(
    f"""
    SELECT *
    FROM nswsc
    ORDER BY embedding_vector ANN OF {embedding} LIMIT 5;
    """
    )
#display(query)

results = session.execute(query)
top_5_products = results._current_rows

for row in top_5_products:
  print(f"""{row.document}\n\n""")

. The
sample was analysed and found to contain less than 0.05 mg of cocaine per litre
of blood.
	The respondent was interviewed by police at the hospital. He stated that he
recalled travelling through the city and going over
the Harbour Bridge. He
stated that he could not recall the events leading up to the collision. He
declined to be interviewed further
upon legal advice.
	Judith Perl, a pharmacologist employed in the Clinical Forensic Unit of the
NSW Police Service for 13 years, expressed the opinion
that at the time of
driving the respondent would have most likely been under the influence of
cocaine to the extent that his driving
ability would have been impaired. Her
opinion is based on the blood sample, the statement to medical staff at the
hospital and details
of the collision provided to her by police. Ms Perl
indicated that cocaine is a central nervous system stimulant which impairs



I do accept, however, that he had used cocaine, either that day or
sufficiently close to the 

In [ ]:
query = SimpleStatement(
    f"""
    SELECT *
    FROM products_table
    ORDER BY openai_description_embedding ANN OF {embedding} LIMIT 5;
    """
    )
#display(query)

results = session.execute(query)
top_5_products = results._current_rows

for row in top_5_products:
  print(f"""{row.product_id}, {row.product_name}, {row.description}\n""")

## Ask ChatGPT for some help

- Here we build a prompt with which we'll query ChatGPT.  Note the "roles" in this little conversation give the LLM more context about who that part of the conversation is coming from.
- This may take 10-20 seconds to return, so be patient.

In [ ]:

message_objects = []

# With the role as 'system',  we tell the model how we want it to behave and tell it how its personality and type of response should be.
message_objects.append({"role":"system",
                        "content":"You're a chatbot helping customers with questions and helping them with product recommendations"})


# With the role as 'user',  pass the question from user.
message_objects.append({"role":"user",
                        "content": customer_input})

message_objects.append({"role":"user",
                        "content": "Please give me a detailed explanation of your recommendations"})

message_objects.append({"role":"user",
                        "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})

message_objects.append({"role":"user",
                        "content":"The computer component itself should be one from the recommended products I will provide"})


# With the role as 'assistant',  load the results from Astra with Vector Search.  That helps the model to provide answer to the question asked by user.
message_objects.append({"role": "assistant",
                        "content": "I found these 5 products I would recommend"})

products_list = []

for row in top_5_products:
    brand_dict = {'role': "assistant", "content": f"{row.description}"}
    products_list.append(brand_dict)

message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)
print(completion.choices[0].message['content'])